In [ ]:
import spacy
import json 

from gensim.models import Word2Vec
import gensim 
from gensim.utils import simple_preprocess



In [6]:
stopwords = []
final = []
for post in posts:
    post = simple_preprocess(post, deacc = True)
    # filter posts less than 25 words long 
    if len(post) > 25:
        post = [token for token in post if str(token) not in stopwords ]
        final.append(post)

# Tf-IDF REMOVAL

___

We arn't going to want to move forward with this kind of word removal until we have done more manual inspection and stop word removal. The brevity of our texts may demand that we keep as many words as possible, and lossing frequently occurring words seminal to the focus of the study .i.e homelessness would prevent us from identifying a posts relevancy overall at this time. for example. we still do not currently know if all of our posts pertain to homelessness, or if they include many discussions of cats up tree's. Untill we have done more cleaning, we will want our topic model to to isolate irrelavant posts, removing "homeless" from all of the posts would make it rather difficult to do this. So it is a step better saved for latter.

In [23]:
# from gensim.models import TfidfModel

# # create word dictionary 
# id2word = corpora.Dictionary(data_bigrams_trigrams)
# # just to make it simpler going forward 
# texts = data_bigrams_trigrams
# # convert all of our texts into a bag of words
# corpus = [id2word.doc2bow(text) for text in texts]

# print ( corpus[0][0:20])

# # instantiate tfidf model 
# tfidf = TfidfModel(corpus, id2word=id2word)

# low_value = 0.03
# words = []
# words_missing_in_tfidf = []


# for i in range(0, len(corpus)):
#     bow = corpus[i]
#     low_value_words = [] 
#     tfidf_ids = [id for id, value in tfidf[bow]]
#     bow_ids = [id for id, value in bow]
#     low_value_words = [id for id, value in tfidf[bow] if value < low_value] 
#     drops = low_value_words+words_missing_in_tfidf
#     for item in drops:
#         words.append(id2word[item])
#     words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # the words with tf-idf score will be missing
    
#     new_bow= [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
#     corpus[i] = new_bow
    
# corpus

# Bigrams &  Trigrams 

___

We attempt to capture some of the more important word pairings with bigrams and trigrams. 

In [22]:
# https://www.youtube.com/watch?v=UEn3xHNBXJU
# https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#9createbigramandtrigrammodels

## set parameters

# def make_bigrams(texts):
#     return [bigram[doc] for doc in texts]

# def make_trigrams(texts):
#     return [trigram[bigram[doc]] for doc in texts]


# # Build the bigram and trigram models
# bigram_model = gensim.models.Phrases(data_words, min_count = 3, threshold = 50 )# min freq for a coupling to be a bigram ## thresh = num of bigrams allowes
# # of the bigrams, are is their overlap in the rest of our words for a trigram?
# trigram_model = gensim.models.Phrases(bigram_model[data_words], threshold = 50 )

## create 

# # Faster way to get a sentence clubbed as a trigram/bigram

# # fit bigram model 
# bigram = gensim.models.phrases.Phraser(bigram_model)
# trigram = gensim.models.phrases.Phraser(trigram_model)

# # instantia
# data_bigrams = make_bigrams(data_words)
# data_bigrams_trigrams = make_trigrams(data_bigrams)

# print(data_bigrams_trigrams)

# Lemma_

___

In [ ]:
def lemmatization(texts, allowed_postages=['NOUN','ADJ','VERB','ADV']):
    
    '''
    allowed_postage : the parts of speach we want to keep [DEFAULT: 'NOUN','ADJ','VERB','ADV'] 
    '''
    
    # load in spacy sm web model 
    nlp = spacy.load('en_core_web_sm', diasble = ['parser','ner']) # computaltionally expensize aspects 
    texts_out = [] # output
    
    # for each post in the corpus
    # iterate over texts
    for text in texts:  
        # creates spacy doc object containing vectorized contextual information like Parts of Speech (pos) 
        doc = nlp(text)
        # list for holding lemmatized tokens
        new_text = []
        # iterate over each token
        for token in doc:
            # only keep the desired pos
                if token.pos_ in allowed_postages and not token.is_stop and not token.is_punct and not token.like_num:
                    # if str(token) not in stopwords:
                        # reducing model complexity by reducing tokens to lemma_ 
                        new_text.append(token.lemma_)   
                        # print(token.lemma_)

        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)

# Similar words

In [17]:
# train W2v with json file 
model = Word2Vec(segments, min_count=1)

In [18]:
# load model 
model = Word2Vec.load('../models/demo.bin')

In [20]:
def sim_word(kwds, top_n):
    """
    kwds : list of search terms
    top_n: number of similar term to return 
    
    returns: top_n most similar words for each of the terms in kwds
    """
    # for each search term 
    for kwd in kwds:
        try:
            print(kwd.upper()) # show search term 
            res = model.wv.similar_by_word(kwd, topn=top_n) # find similar words
            for term in res:
                print(term) # show similar words
        except KeyError:
            print('[]"Word not in vocabulary"[]]')
            continue
    

In [ ]:
def gen_similarity(word, filepath):
    model = KeyedVectors.load_word2vec_format(f'{filepath}', binary = False)
    results = model.most_similar(positive =[word])
    print(results)

In [30]:
gen_similarity('homeless','../data/word_vectors_demo_0.0.txt')

[('shelter', 0.9994912147521973), ('downtown', 0.9993812441825867), ('those', 0.999374508857727), ('coalition', 0.9993677139282227), ('shelters', 0.9993603229522705), ('coliseum', 0.9993539452552795), ('hosting', 0.9993459582328796), ('bbq', 0.9993423223495483), ('monthly', 0.9993346333503723), ('campsite', 0.9993331432342529)]


## For creating a labeled dataset

In [24]:
def cat_posts(post_tokens, kwds, label):    
    '''
    label: class label 
    kwds: search terms
    post_tokens: tokenized corpus
    
    returns: labeled dataset containing target posts
    '''
    
    train_data = []   
    match = None
    # if a post has one of our kwds
    for word in kwds:
        for post in post_tokens:
            match = False 
            # only relevant posts 
            if word in post:
                match = True
            # if our keyword is in a post then match = True    
            # attaches given class label to post
            if match == True:
                post = ' '.join(post)
                train_data.append((post,label))

    return train_data

In [ ]:
import json 
from gensim.models.word2vec import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
import multiprocessing  # for using seperate core's

In [ ]:
def training(model_name, filepath):
    with open (f'{filepath}', 'r', encoding = 'utf-8') as f:
        texts = json.load(f)
    sentence = texts 
    cores = multiprocessing.cpu_count()
    w2v_model = Word2Vec(
            min_count = 2,
            window = 2, # num of surrounding words to consider 
            # shape of a word vector
            size = 500,  #size of vocab , dimensions of vocab                  
            sample = 6e-5,
            alpha =0.03, # error term
            min_alpha=0.0007, # ??
            negative = 20, # ??
            workers = cores-1 # number of cores to train with  
        )
    
    w2v_model.build_vocab(texts) # create the model vocabulary 
    w2v_model.train(texts, total_examples=w2v_model.corpus_count, epochs = 30) # train the model
    w2v_model.save(f'../data/word_vec_model_{model_name}.model') # save the model for comparison 
    w2v_model.wv.save_word2vec_format(f'../data/word_vectors_{model_name}.txt') # save word vectors
    # https://www.youtube.com/watch?v=1  